Usiamo i dati di un periodo passato per trovare i pesi ottimali di Markowitz (il Tangency Portfolio) e poi vediamo come si sarebbero comportati nel 2024.

In [ ]:
!pip install yfinance

import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

In [ ]:
# Download dati
# Definisco gli asset: Azioni USA, Oro, Obbligazioni 20+ anni
tickers = ["SPY","GLD","GOOGL"]
data = yf.download(tickers,
                   start="2022-01-01",
                   end="2024-12-31")["Close"]

returns = np.log(data/data.shift(1)).dropna()

In [ ]:
# Divido il dataset in dati "in sample" e "out of sample"
train_returns = returns.loc["2022-01-01":"2023-12-31"]
test_returns = returns.loc["2024-01-01":]

In [ ]:
# Definisco la funzione per massimizzare lo Sharpe ratio
def get_ret_vol_sharpe(weights,returns):
  weights = np.array(weights)
  ret = np.sum(returns.mean()*weights)*252
  vol = np.sqrt(np.dot(weights.T,np.dot(returns.cov()*252,weights)))
  sr = ret/vol  # Assumiamo rf=0 per semplicità
  return np.array([ret,vol,sr])

In [ ]:
# Funzione obiettivo da minimizzare (minimizziamo l'opposto dello sr)
def neg_sharpe(weights,returns):
  return get_ret_vol_sharpe(weights,returns)[2] * -1

In [ ]:
# Ottimizzo per trovare i pesi (basandomi solo sul sample)
cons = ({"type":"eq","fun":lambda x: np.sum(x)-1}) # somma dei pesi = 1
bounds = tuple((0.10, 0.60) for i in range(len(tickers))) # Facciamo in modo che gli asset possano pesare dal 10% al 60% per evitare troppa concentrazione in un unico asset, cosa che potrebbe verificarsi nel modello di Markowitz.
init_guess = [1/len(tickers) for i in range(len(tickers))] # Pesi uguali

opt_results = minimize(neg_sharpe,init_guess, args=(train_returns),
                       method="SLSQP",bounds=bounds,constraints=cons)
tangency_weights = opt_results.x
print(f"Pesi ottimali calcolati nel training: {dict(zip(tickers, tangency_weights.round(4)))}")



In [ ]:
# Uso i pesi ottimali vincolati sui dati del 2024

# Assicuriamoci che i pesi siano un array piatto
weights_arr = np.array(tangency_weights).flatten()

portfolio_test_returns = pd.Series(test_returns.values @ weights_arr, index=test_returns.index)

# Creiamo un benchmark "Naive" (Equally Weighted: 33.3% ciascuno)
equal_weights = np.array([1/len(tickers)] * len(tickers))
benchmark_returns = pd.Series(test_returns.values @ equal_weights, index=test_returns.index)

In [ ]:
# Performance cumulata
# Trasformo i rendimenti in un indice che parte da 1
cum_port = np.exp(portfolio_test_returns.cumsum())
cum_bench = np.exp(benchmark_returns.cumsum())

In [ ]:
# Calcolo le metriche per confrontare i dati


# Rendimento totale nel 2024
total_ret_port = (cum_port.iloc[-1] - 1) * 100
total_ret_bench = (cum_bench.iloc[-1] - 1) * 100


# Volatilità annualizzata
vol_port = portfolio_test_returns.std() * np.sqrt(252)
vol_bench = benchmark_returns.std() * np.sqrt(252)


# Calcolo Sharpe ratio
sharpe_port = (portfolio_test_returns.mean() * 252) / vol_port
sharpe_bench = (benchmark_returns.mean() * 252) / vol_bench


print(f"--- RISULTATI BACKTEST (2024) ---")
print(f"MARKOWITZ:")
print(f"  Rendimento Totale: {total_ret_port:.2f}%")
print(f"  Volatilità Annua:  {vol_port:}%")
print(f"  Sharpe Ratio:      {sharpe_port:}")

print(f"\nBENCHMARK (1/N):")
print(f"  Rendimento Totale: {total_ret_bench:.2f}%")
print(f"  Volatilità Annua:  {vol_bench:}%")
print(f"  Sharpe Ratio:      {sharpe_bench:}")


